In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
from torch import optim
from torch.nn import functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import PIL
from torch import nn
device = "cuda" if torch.cuda.is_available() else "cpu"
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
from landmark_crop import FaceCropper
class dataset(Dataset):
    def __init__(self, df, face_cropper):
        self.df = df
        self.normalize = transforms.Normalize(mean = [0.485,0.456,0.406,0.485,0.456,0.406],std = [0.229,0.224,0.225, 0.229,0.224,0.225])

        self.face_cropper = face_cropper

    def __len__(self):
        return len(self.df)

    def __getitem__(self, ix):
        file = self.df.iloc[ix]
        age = torch.tensor(file.age / 116)
        gender = torch.tensor(file.gender).view(-1)
        eth = torch.tensor(file.ethnicity)
        im = file.pixels
        im = np.array(Image.fromarray(im).convert("RGB"))
        print(im.shape)

        # Use the FaceCropper to crop faces before normalization
        cropped_faces = self.face_cropper.crop_faces_and_concat(im, mask=False)
        print(cropped_faces.shape)

        # Assuming you want to use the first cropped face (you can modify this as needed)
        im = cropped_faces

        # im = cv.resize(im, (224, 224))
        # im = torch.tensor(im).permute(2, 1, 2)
        # im = torch.tensor(im).permute(2,0,1)
        im = self.normalize(im/255)


        return im.float().to(device), age.to(device), gender.float().to(device), eth.to(device)


In [1]:
import torch
import torch.nn as nn
from torchvision import models

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()

        # Load VGG16 backbone with pretrained weights
        self.vgg16 = models.vgg16(pretrained=True)

        # Modify input layer to accept 6 channels
        self.vgg16.features[0] = nn.Conv2d(6, 64, kernel_size=(3, 3), padding=(1, 1), bias=False)

        # Freeze backbone layers for transfer learning
        for param in self.vgg16.parameters():
            param.requires_grad = False

        # Define output layers
        self.age_classifier = nn.Sequential(
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
        self.gender_classifier = nn.Sequential(
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
        self.eth_classifier = nn.Sequential(
            nn.Linear(64, 5),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.vgg16.features(x)
        x = x.view(x.size(0), -1)  # Flatten

        # Add a new layer to reduce features to 64
        x = nn.Linear(25088, 64)(x)

        age_output = self.age_classifier(x)
        gender_output = self.gender_classifier(x)
        eth_output = self.eth_classifier(x)

        return age_output, gender_output, eth_output
    
model = CustomModel()
# Input tensor with shape (batch_size, channels, height, width)
input_tensor = torch.randn(32, 6, 244, 244)

# Pass input through model
age_output, gender_output, eth_output = model(input_tensor)


/home/niran/miniconda3/envs/deepface/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/niran/miniconda3/envs/deepface/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/niran/miniconda3/envs/deepface/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
